# Activity Tracking with APOLLO

## CONTINUE NOTE
Take notes on activities on [Notion](https://www.notion.so/7cups/Window-activity-tracking-desktop-phone-99b27dc20ac2429ea7f5b49958ca4d2d#0655378b1b1e4ef2ab051290e26d4293) and commit changes in git

In [69]:
# To use 100% width!
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/fk/sjtjy3d1437533rdvljmg56h0000gn/T/ipykernel_3414/485875232.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [52]:
import os
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', 0)
datadir = lambda filename: os.path.join('USER_DATA', filename)

In [49]:
df = pd.read_csv(datadir('apollo_2.csv'), on_bad_lines='skip', delimiter='\t')
df.Timestamp = pd.to_datetime(df.Timestamp)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53489 entries, 0 to 53488
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  53489 non-null  datetime64[ns]
 1   Activity   53489 non-null  object        
 2   Output     53489 non-null  object        
 3   Database   53489 non-null  object        
 4   Module     53489 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.0+ MB


In [53]:
df.Output

0        [START: 2022-01-23 17:11:05] [END: 2022-01-23 20:58:25] [USAGE IN SECONDS: 13639.929563045502] [USAGE IN MINUTES: 227.33215938409168] [STREAM NAME: /knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78] [STRING: Rapport] [VALUE: 4.096694909699153e+18] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 20:58:26] [UUID: 737A86AE-C7F0-4ED6-BA81-43CE4D6EF8EE] [ZOBJECT TABLE ID: 877016]    
1        [START: 2022-01-23 09:06:00] [END: 2022-01-23 17:11:05] [USAGE IN SECONDS: 29104.622684001923] [USAGE IN MINUTES: 485.07704473336537] [STREAM NAME: /knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78] [STRING: Rapport] [VALUE: 4.096694909699153e+18] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 20:58:27] [UUID: B091B413-FBE2-4667-B182-10C8A314A65B] [ZOBJECT TABLE ID: 877167]    
2        [START: 2022-01-18 05:07:50] [END: 2022-01-18 14:45:31] [USAGE IN SECONDS: 34661.0360070467] [USAGE IN MINUTES: 577.6839334507783] [S

In [59]:
import re

In [70]:
previewdf = lambda df: display(HTML(df.to_html()))
previewseries = lambda series: display(HTML(pd.DataFrame(series).to_html()))

In [86]:
class Crawler:
    apollo_cols = ['Timestamp', 'Activity', 'Output', 'Database', 'Module']
    ts = 'Timestamp'
    def __init__(self, df):
        self.row_count = 0
        self.df = df
        self.activities = df.Activity.unique().tolist()
        self.tables = {act: None for act in self.activities}
    
    def get_output_df(self, output_series):
        out = output_series
        out_dict = {}
        all_keys = set()
        out_len = 0
        for row in out:
            attributes = re.findall('\[+(.*?)\]', row)
            attr_dict = {}
            for attr in attributes:
                tmp = attr.split(':')
                key = tmp[0].strip()
                val = tmp[1].strip()
                attr_dict[key] = val
                
            all_keys = all_keys.union(set(attr_dict.keys()))
            for key in all_keys:
                if key not in out_dict:
                    out_dict[key] = ['']*out_len
                if key in attr_dict:
                    out_dict[key].append(attr_dict[key])
                else:
                    out_dict[key].append('')
            
#             print(out_dict)
#             out_df = pd.DataFrame(out_dict)
#             previewdf(out_df)
            out_len += 1
        out_df = pd.DataFrame(out_dict)
        return out_df
    
    def create_table_for_(self, activity):
        print("Creating table for activity:", activity)
        actdf = self.df[self.df.Activity == activity].reset_index(drop=True)

        out_df = self.get_output_df(actdf.Output)
        out_df.loc[:, self.ts] = actdf[self.ts]
#         previewdf(out_df.tail(10))
        print("Done | Shape:", out_df.shape, "| Columns:", out_df.columns)
        return out_df
    
    def get_tables(self):
        for act in self.activities:
            self.tables[act] = self.create_table_for_(act)
        return self.tables
    
cr = Crawler(df)
tables = cr.get_tables()
tables.keys()

Creating table for activity: Knowledge Sync Deletion Bookmark
Done | Shape: (368, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Knowledge Sync Addition Window
Done | Shape: (736, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Application Intents
Done | Shape: (4130, 20) | Columns: Index(['ITEM ID', 'START', 'END', 'GROUP ID', 'BUNDLE ID', 'GMT OFFSET',
       'ZOBJECT TABLE ID', 'DAY OF WEEK', 'HANDLING STATUS', 'UUID',
       'SERIALIZED INTERACTION (HEX)', 'ZMETADATAHASH', 'DEVICE ID',
       'APP NAME', 'SOURCE ID', 'INTENT VERB', 'INTENT CLASS',
     

dict_keys(['Knowledge Sync Deletion Bookmark', 'Knowledge Sync Addition Window', 'Application Intents', 'Now Playing', 'Bluetooth Connected', 'Device Lock Status', 'Portrait Topic', 'Backlight Status', 'Application Web Usage', 'Portrait Entity', 'Notification Usage', 'Application Usage', 'Event Tombstone'])

In [56]:
df.Activity.unique().tolist()

['Knowledge Sync Deletion Bookmark',
 'Knowledge Sync Addition Window',
 'Application Intents',
 'Now Playing',
 'Bluetooth Connected',
 'Device Lock Status',
 'Portrait Topic',
 'Backlight Status',
 'Application Web Usage',
 'Portrait Entity',
 'Notification Usage',
 'Application Usage',
 'Event Tombstone']

In [14]:
df.Activity.value_counts(dropna=False)

Application Usage                   10418
Application Web Usage               10306
Notification Usage                  10071
Portrait Topic                       6111
Application Intents                  4130
Portrait Entity                      3649
Backlight Status                     3457
Device Lock Status                   3263
Knowledge Sync Addition Window        736
Event Tombstone                       622
Knowledge Sync Deletion Bookmark      368
Bluetooth Connected                   305
Now Playing                            53
Name: Activity, dtype: int64

## YOLO

In [38]:
yolo = pd.read_csv(datadir('apollo_3.csv'), on_bad_lines='skip', delimiter='\t')
yolo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89109 entries, 0 to 89108
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  89109 non-null  object
 1   Activity   89109 non-null  object
 2   Output     89109 non-null  object
 3   Database   89109 non-null  object
 4   Module     89109 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB


In [87]:
yolo_cr = Crawler(yolo)
yolo_tables = yolo_cr.get_tables()
yolo_tables.keys()

Creating table for activity: Knowledge Sync Deletion Bookmark
Done | Shape: (372, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Knowledge Sync Addition Window
Done | Shape: (744, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Application Intents
Done | Shape: (8260, 25) | Columns: Index(['ITEM ID', 'START', 'END', 'GROUP ID', 'BUNDLE ID', 'INTENT TYPE',
       'GMT OFFSET', 'ZOBJECT TABLE ID', 'DAY OF WEEK', 'HANDLING STATUS',
       'DIRECTION', 'UUID', 'DERIVED INTENT ID',
       'SERIALIZED INTERACTION (HEX)', 'DEVICE ID', 'ZMETADATAHASH',
       'APP 

dict_keys(['Knowledge Sync Deletion Bookmark', 'Knowledge Sync Addition Window', 'Application Intents', 'Now Playing', 'Bluetooth Connected', 'App Media Usage', 'Device Lock Status', 'Portrait Topic', 'Backlight Status', 'Application Web Usage', 'Portrait Entity', 'Notification Usage', 'Application Usage', 'Device Discoverability', 'Event Tombstone'])

In [88]:
yolo[yolo.Activity == 'App Media Usage']

,Timestamp,Activity,Output,Database,Module
9416,2022-01-23 20:59:49,App Media Usage,[START: 2022-01-23 20:59:49] [END: 2022-01-23 20:59:54] [USAGE IN SECONDS: 5] [USAGE IN MINUTES: 0.08333333333333333] [BUNDLE ID: com.apple.quicklook.QuickLookUIService] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 20:59:54] [UUID: 7FBBF841-4E9F-4549-B0D5-D7E4360DA844] [ZOBJECT TABLE ID: 877175],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9417,2022-01-23 20:59:57,App Media Usage,[START: 2022-01-23 20:59:57] [END: 2022-01-23 21:00:00] [USAGE IN SECONDS: 3] [USAGE IN MINUTES: 0.05] [BUNDLE ID: com.apple.quicklook.QuickLookUIService] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 21:00:00] [UUID: 74EEB3EB-77F2-4654-9F3F-2E6BAF2114F1] [ZOBJECT TABLE ID: 877176],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9418,2022-01-24 00:56:52,App Media Usage,[START: 2022-01-24 00:56:52] [END: 2022-01-24 00:57:14] [USAGE IN SECONDS: 22] [USAGE IN MINUTES: 0.36666666666666664] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 00:57:14] [UUID: C170096D-E944-46F8-89E1-067AD722BA3A] [ZOBJECT TABLE ID: 878824],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9419,2022-01-24 04:03:16,App Media Usage,[START: 2022-01-24 04:03:16] [END: 2022-01-24 04:04:19] [USAGE IN SECONDS: 63] [USAGE IN MINUTES: 1.05] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 04:04:19] [UUID: DB1986FE-DC7D-441D-94D4-679473390ECA] [ZOBJECT TABLE ID: 879458],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9420,2022-01-24 04:06:06,App Media Usage,[START: 2022-01-24 04:06:06] [END: 2022-01-24 04:06:34] [USAGE IN SECONDS: 28] [USAGE IN MINUTES: 0.4666666666666667] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 04:06:34] [UUID: 6FB421D7-325A-49AD-9C35-C58D27208039] [ZOBJECT TABLE ID: 879464],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
...,...,...,...,...,...
9919,2022-02-20 10:55:20,App Media Usage,[START: 2022-02-20 10:55:20] [END: 2022-02-20 10:55:26] [USAGE IN SECONDS: 6] [USAGE IN MINUTES: 0.1] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:55:26] [UUID: E0189EEC-8863-4EB0-B0F1-DD16DD53B883] [ZOBJECT TABLE ID: 944663],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9920,2022-02-20 10:56:48,App Media Usage,[START: 2022-02-20 10:56:48] [END: 2022-02-20 10:56:48] [USAGE IN SECONDS: 0] [USAGE IN MINUTES: 0.0] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:56:48] [UUID: 53B2CE89-6353-4C2C-ABAB-DC8473D58FF6] [ZOBJECT TABLE ID: 944666],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9921,2022-02-20 10:57:14,App Media Usage,[START: 2022-02-20 10:57:14] [END: 2022-02-20 10:57:32] [USAGE IN SECONDS: 18] [USAGE IN MINUTES: 0.3] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:57:32] [UUID: 6402A980-B2E5-4A85-BDBA-822885A749E1] [ZOBJECT TABLE ID: 944667],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_ap

In [89]:
set(yolo_tables.keys()) - set(tables.keys())

{'App Media Usage', 'Device Discoverability'}

In [115]:
yolo_tables['Device Discoverability'].sample(10)

,START,UUID,ENTRY CREATION,OS BUILD,ZMETADATAHASH,END,APP NAME,ZBUNDLEID,GMT OFFSET,DISCOVERABILITY SIGNAL CONTEXT,ZOBJECT TABLE ID,DAY OF WEEK,Timestamp
7268,2021-08-30 15,6FC710CB-78EB-44B9-979C-817747805193,2021-08-30 15,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 15,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,640755,Monday,2021-08-30 15:05:47
5590,2021-08-30 13,AF2C32A2-8ED2-4FDB-A7E1-366D8E4D1102,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,639073,Monday,2021-08-30 13:41:26
8716,2021-10-26 21,CEE22262-55D5-4E18-BF3C-E99DE8288BBB,2021-10-26 21,macOS-21A5506j,197bf0975bcb035e01a0f515ea6985bb,2021-10-26 21,com.apple.notificationcenter.opened,com.apple.notificationcenterui,-4,gesture,722354,Tuesday,2021-10-26 21:37:18
8005,2021-09-20 05,C771F933-25D0-4157-A74F-4DC019F6AB6B,2021-09-20 05,macOS-21A5294g,447797f980c6082c8142042f6d38bd0a,2021-09-20 05,com.apple.notificationcenter.opened,com.apple.notificationcenterui,-4,gesture,666677,Monday,2021-09-20 05:28:34
5786,2021-08-30 13,F4C466CC-069F-48CF-ACC0-4FE9A6D3640C,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,639269,Monday,2021-08-30 13:50:50
5497,2021-08-30 13,651C3CB4-B24A-41B8-BEDC-B3E5CD14E2C0,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,638980,Monday,2021-08-30 13:36:31
1094,2021-08-30 09,8F9404A9-C92F-4808-9026-391D9054E021,2021-08-30 09,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 09,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,634577,Monday,2021-08-30 09:53:27
2793,2021-08-30 11,853563A9-D4C8-4826-8F2A-1F58FB117359,2021-08-30 11,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 11,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,636276,Monday,2021-08-30 11:19:39
4137,2021-08-30 12,4AA999A8-39DA-4FBC-82AC-B616EF4B1C2E,2021-08-30 12,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 12,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,637620,Monday,2021-08-30 12:27:43
2633,2021-08-30 11,5ADB1930-6A91-4D11-B9ED-0CBF0BD84B1D,2021-08-30 11,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 11,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,636116,Monday,2021-08-30 11:11:32
